In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime

In [2]:
selected_df = pd.read_csv('./국토교통부_실거래가_공개시스템/집값분석/일괄구매/일괄구매_전체.csv', index_col=[0])
print(selected_df.shape)
selected_df.head()

(44864, 15)


,지번주소,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일,단가평균,구,동,동2
0,서울특별시 강남구 개포동 1169-10,에이스빌라,36.46,20.40,10500,2,2001.0,6.0,2007-04-10,287.986835,NaN,288.782553,강남구,개포동,강남구_개포동
1,서울특별시 강남구 개포동 1169-10,에이스빌라,36.46,20.40,10500,3,2001.0,6.0,2007-04-10,287.986835,NaN,288.782553,강남구,개포동,강남구_개포동
2,서울특별시 강남구 개포동 1169-10,에이스빌라,30.00,16.78,8500,2,2001.0,6.0,2007-04-10,283.333333,NaN,288.782553,강남구,개포동,강남구_개포동
3,서울특별시 강남구 개포동 1169-10,에이스빌라,63.15,35.33,19000,4,2001.0,6.0,2007-04-10,300.870942,NaN,288.782553,강남구,개포동,강남구_개포동
4,서울특별시 강남구 개포동 1169-10,에이스빌라,30.00,16.78,8500,3,2001.0,6.0,2007-04-10,283.333333,NaN,288.782553,강남구,개포동,강남구_개포동


In [7]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [4]:
def date_modifier(x):
    # x is a string or datetime
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if '-' in x:
            x = x.replace('-', '.')
            
        splitted = x.split('.')
        
        if len(splitted) == 1:
            return np.nan
        else:
            return x.split(' ')[0]

In [5]:
def get_year(x):
    if pd.isna(x) == True:
        return x
    else:
        return int(x.split('.')[0])

In [10]:
reference_df = pd.read_excel('./감정평가사_자료/이승준_총괄표.xlsx', header=[0,1]).dropna(subset=[('소재지','소재지')]).sort_values([('소재지', '지역'),('소재지', '소재지'), ('소재지', '지번')]).reset_index(drop=True)
reference_df.columns = [col[0] + '_' + col[1] for col in reference_df.columns]
reference_df['소재지_지번'] = reference_df['소재지_지번'].apply(landnum_modifier)
reference_df['지번주소'] = '서울특별시 ' + reference_df['소재지_지역'] + ' ' + reference_df['소재지_소재지']\
+ ' ' + reference_df['소재지_지번']
reference_df['기준시점2'] = reference_df['수입/비용_기준시점'].apply(date_modifier)
reference_df['년'] = reference_df['기준시점2'].apply(get_year)
#reference_df = reference_df.dropna(subset=['기준시점2'])
reference_df = reference_df.reset_index(drop=True)
print(reference_df.shape)
reference_df.head()

(131, 42)


,소재지_지역,소재지_소재지,소재지_지번,토지내용_용도지역,토지내용_도로너비,토지내용_토지면적(㎡),토지내용_토지매매금액,토지내용_토지매매단가\n(원/㎡),토지내용_공시지가와격차\n(개공/매매),건물내용_건물용도,건물내용_공급면적(㎡),건물내용_전유면적(㎡),건물내용_전용율,건물내용_용적율\n(%),건물내용_건폐율\n(%),건물내용_구분건물호,건물내용_건물동수,건물내용_건물층수,수입/비용_의뢰인,수입/비용_기준시점,수입/비용_전체평가금액,수입/비용_투입비용합계,수입/비용_순이익,수입/비용_순이익율,평가단가(원/전유㎡) _전유면적,평가단가(원/전유㎡) _공급면적,매출원가(원/전유㎡) _전유면적기준,매출원가(원/전유㎡) _공급면적기준,사업진행일자_토지계약일자,사업진행일자_허가일자,사업진행일자_착공일자,사업진행일자_사용승인일,건축비\n(원/평)_Unnamed: 32_level_1,원가비율_토지원가비율,원가비율_건축원가비율,원가비율_부대비용,원가비율_금융비용,원가비율_합계,비고_Unnamed: 38_level_1,지번주소,기준시점2,년
0,강남구,개포동,1195-10,2종일주,6미터,265.7,2.970000e+09,11178020.0,0.433,다세대주택,519.54,432.22,0.832,195.54,59.62,15개호,1개동,지상5층,SH공사,2020.09.23,4.719500e+09,4.167814e+09,5.516858e+08,0.117,10919208.0,9.083997e+06,9.642807e+06,8372395.0,2019.02.26,2019.06.18,2019.06.18,2020.01.13,5500000.0,0.7563,0.2139,0.0060,0.0239,1.0,NaN,서울특별시 강남구 개포동 1195-10,2020.09.23,2020.0
1,강남구,개포동,1199-7,2종일주,4미터,248.0,2.550000e+09,10282258.0,0.365,다세대주택,463.84,390.67,0.842,187.04,59.61,14개호,1개동,지상5층,LH공사,2019.11.29,4.040000e+09,3.593249e+09,4.467511e+08,0.111,10341209.0,8.709524e+06,9.197658e+06,8074624.0,2018.08.21,2018.12.31,2019.01.07,2019.06.04,5400000.0,0.7520,0.2175,0.0061,0.0244,1.0,NaN,서울특별시 강남구 개포동 1199-7,2019.11.29,2019.0
2,강남구,개포동,1216-7,2종일주,8미터,258.1,2.931000e+09,11356064.0,0.472,도시행생활주택,489.46,400.29,0.818,199.62,59.89,15개호,1개동,지상5층,SH공사,2019.11.29,4.586000e+09,4.054965e+09,5.310348e+08,0.116,11456694.0,9.369509e+06,1.013007e+07,8312283.0,2019.01.24,2019.06.17,2019.07.23,2020.02.25,5400000.0,0.7667,0.2034,0.0057,0.0242,1.0,NaN,서울특별시 강남구 개포동 1216-7,2019.11.29,2019.0
3,강남구,역삼동,707-9,일반상업,8미터,363.5,9.500000e+09,26134801.0,0.580,도시행생활주택,1695.85,1289.62,0.760,199.62,59.89,15개호,1개동,지상5층,LH공사,2019.11.29,2.219983e+10,1.398811e+10,8.211721e+09,0.370,17214239.0,1.115945e+07,1.084669e+07,16602582.0,2019.08.14,2019.07.15,2020.01.03,2020.10.07,5800000.0,0.7189,0.2568,0.0072,0.0170,1.0,NaN,서울특별시 강남구 역삼동 707-9,2019.11.29,2019.0
4,강남구,역삼동,751-6,2종일주,6미터,391.8,5.900000e+09,15058703.0,0.441,도시행생활주택,658.88,537.65,0.816,168.17,46.35,20개호,1개동,지상6층,LH공사,2019.11.29,9.810500e+09,7.604511e+09,2.205989e+09,0.225,18247001.0,1.488966e+07,1.414398e+07,12107552.0,2019.01.04,2019.03.05,2019.04.20,2019.11.29,5500000.0,0.8249,0.1486,0.0042,0.0223,1.0,NaN,서울특별시 강남구 역삼동 751-6,2019.11.29,2019.0


In [12]:
selected_addr_df

,지번주소,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일,단가평균,구,동,동2
57,서울특별시 강남구 개포동 1195-10,동영빌,33.71,20.72,36150,5,2020.0,0.0,2020-03-02,1072.382082,NaN,1097.178201,강남구,개포동,강남구_개포동
58,서울특별시 강남구 개포동 1195-10,동영빌,25.55,15.71,28500,2,2020.0,0.0,2020-03-02,1115.459883,NaN,1097.178201,강남구,개포동,강남구_개포동
59,서울특별시 강남구 개포동 1195-10,동영빌,25.39,15.61,28300,2,2020.0,0.0,2020-03-02,1114.612052,NaN,1097.178201,강남구,개포동,강남구_개포동
60,서울특별시 강남구 개포동 1195-10,동영빌,29.56,18.17,32950,2,2020.0,0.0,2020-03-02,1114.682003,NaN,1097.178201,강남구,개포동,강남구_개포동
61,서울특별시 강남구 개포동 1195-10,동영빌,25.46,15.65,28400,2,2020.0,0.0,2020-03-02,1115.475255,NaN,1097.178201,강남구,개포동,강남구_개포동
62,서울특별시 강남구 개포동 1195-10,동영빌,26.21,16.11,29200,2,2020.0,0.0,2020-03-02,1114.078596,NaN,1097.178201,강남구,개포동,강남구_개포동
63,서울특별시 강남구 개포동 1195-10,동영빌,26.21,16.11,29200,3,2020.0,0.0,2020-03-02,1114.078596,NaN,1097.178201,강남구,개포동,강남구_개포동
64,서울특별시 강남구 개포동 1195-10,동영빌,25.55,15.71,28500,3,2020.0,0.0,2020-03-02,1115.459883,NaN,1097.178201,강남구,개포동,강남구_개포동
65,서울특별시 강남구 개포동 1195-10,동영빌,25.39,15.61,28300,3,2020.0,0.0,2020-03-02,1114.612052,NaN,1097.178201,강남구,개포동,강남구_개포동
66,서울특별시 강남구 개포동 1195-10,동영빌,29.56,18.17,32950,3,2020.0,0.0,2020-03-02,1114.682003,NaN,1097.178201,강남구,개포동,강남구_개포동


In [23]:
for i in range(reference_df.shape[0]):
    addr = reference_df.loc[i, '지번주소']
    print('------------------')
    print(addr)
    selected_addr_df = selected_df[selected_df['지번주소'] == addr]
    print('평가 호 수:', reference_df.loc[i, '건물내용_구분건물호'])
    print('실거래 호 수:', selected_addr_df.shape[0])
    print('평가건물용도:', reference_df.loc[i, '건물내용_건물용도'])
    print('평가금액:', reference_df.loc[i, '수입/비용_전체평가금액'] / 10000)
    print('실거래금액:', selected_addr_df['거래금액(만원)'].sum())
    print('------------------')

------------------
서울특별시 강남구 개포동 1195-10
평가 호 수: 15개호
실거래 호 수: 15
평가건물용도: 다세대주택
평가금액: 471950.0
실거래금액: 471950
------------------
------------------
서울특별시 강남구 개포동 1199-7
평가 호 수: 14개호
실거래 호 수: 14
평가건물용도: 다세대주택
평가금액: 404000.0
실거래금액: 404000
------------------
------------------
서울특별시 강남구 개포동 1216-7
평가 호 수: 15개호
실거래 호 수: 15
평가건물용도: 도시행생활주택
평가금액: 458600.0
실거래금액: 458600
------------------
------------------
서울특별시 강남구 역삼동 707-9
평가 호 수: 15개호
실거래 호 수: 0
평가건물용도: 도시행생활주택
평가금액: 2219982.75
실거래금액: 0
------------------
------------------
서울특별시 강남구 역삼동 751-6
평가 호 수: 20개호
실거래 호 수: 20
평가건물용도: 도시행생활주택
평가금액: 981050.0
실거래금액: 981050
------------------
------------------
서울특별시 강남구 역삼동 778-15
평가 호 수: 29개호
실거래 호 수: 29
평가건물용도: 도생-다세대
평가금액: 1546000.0
실거래금액: 1528000
------------------
------------------
서울특별시 강남구 역삼동 778-3
평가 호 수: 12개호
실거래 호 수: 12
평가건물용도: 도생-원룸
평가금액: 441000.0
실거래금액: 441000
------------------
------------------
서울특별시 강동구 고덕동 292-5
평가 호 수: 10개호
실거래 호 수: 10
평가건물용도: 다세대주택
평가금액: 283950.0
실거래금액: 283950
-

평가 호 수: 18개호
실거래 호 수: 18
평가건물용도: 도생
평가금액: 490350.0
실거래금액: 490350
------------------
------------------
서울특별시 금천구 시흥동 790-8
평가 호 수: 20개호
실거래 호 수: 20
평가건물용도: 도생-단지형
평가금액: 541600.0
실거래금액: 541650
------------------
------------------
서울특별시 금천구 시흥동 811-7
평가 호 수: 24개호
실거래 호 수: 24
평가건물용도: 단지형 다세대
평가금액: 510600.0
실거래금액: 510600
------------------
------------------
서울특별시 금천구 시흥동 815-9
평가 호 수: 28개호
실거래 호 수: 28
평가건물용도: 다세대주택
평가금액: 546850.0
실거래금액: 546850
------------------
------------------
서울특별시 금천구 시흥동 824-16
평가 호 수: 28개호
실거래 호 수: 0
평가건물용도: 다세대주택
평가금액: 800200.0
실거래금액: 0
------------------
------------------
서울특별시 금천구 시흥동 825-19
평가 호 수: 21개호
실거래 호 수: 21
평가건물용도: 다세대주택
평가금액: 626400.0
실거래금액: 626400
------------------
------------------
서울특별시 금천구 시흥동 919-78
평가 호 수: 37개호
실거래 호 수: 67
평가건물용도: 다세대주택
평가금액: 923900.0
실거래금액: 1875700
------------------
------------------
서울특별시 금천구 시흥동 919-88
평가 호 수: 34개호
실거래 호 수: 34
평가건물용도: 다세대주택
평가금액: 912400.0
실거래금액: 912400
------------------
------------------
서울특별시 금천구 시흥동

In [20]:
inv_df = reference_df[reference_df['지번주소'].str.contains('우이동')]
print(inv_df.shape)

(9, 42)


In [21]:
inv_df

,소재지_지역,소재지_소재지,소재지_지번,토지내용_용도지역,토지내용_도로너비,토지내용_토지면적(㎡),토지내용_토지매매금액,토지내용_토지매매단가\n(원/㎡),토지내용_공시지가와격차\n(개공/매매),건물내용_건물용도,건물내용_공급면적(㎡),건물내용_전유면적(㎡),건물내용_전용율,건물내용_용적율\n(%),건물내용_건폐율\n(%),건물내용_구분건물호,건물내용_건물동수,건물내용_건물층수,수입/비용_의뢰인,수입/비용_기준시점,수입/비용_전체평가금액,수입/비용_투입비용합계,수입/비용_순이익,수입/비용_순이익율,평가단가(원/전유㎡) _전유면적,평가단가(원/전유㎡) _공급면적,매출원가(원/전유㎡) _전유면적기준,매출원가(원/전유㎡) _공급면적기준,사업진행일자_토지계약일자,사업진행일자_허가일자,사업진행일자_착공일자,사업진행일자_사용승인일,건축비\n(원/평)_Unnamed: 32_level_1,원가비율_토지원가비율,원가비율_건축원가비율,원가비율_부대비용,원가비율_금융비용,원가비율_합계,비고_Unnamed: 38_level_1,지번주소,기준시점2,년
41,강북구,우이동,100-31,2종일주,6미터,338.0,1.616000e+09,4781065.0,0.493,다세대주택,659.19,579.99,1.270,195.03,58.62,11개호,1개동,지상5층,LH공사,2020.06.18,3.183000e+09,2.585640e+09,597359525.5,0.188,5488026.0,6.968649e+06,4.458078e+06,4662514.0,2019.07.16,2019.09.30,2019.10.25,2020.03.24,5500000.0,0.6654,0.3031,0.0085,0.0231,1.0,NaN,서울특별시 강북구 우이동 100-31,2020.06.18,2020.0
42,강북구,우이동,147-21,2종일주,4미터,222.0,7.800000e+08,3513514.0,0.558,다세대주택,400.77,329.21,0.757,199.84,59.42,10개호,1개동,지상5층,LH공사,2019.01.28,1.775000e+09,1.622029e+09,152970847.5,0.086,5391695.0,4.081773e+06,4.927035e+06,3849610.0,2018.02.05,2018.07.11,2018.07.18,2019.01.16,5400000.0,0.5114,0.4516,0.0126,0.0244,1.0,NaN,서울특별시 강북구 우이동 147-21,2019.01.28,2019.0
43,강북구,우이동,154-37,1종일주,15미터,719.0,2.503000e+09,3481224.0,0.833,다세대주택,1120.03,883.33,0.645,149.65,52.12,31개호,2개동,지상5층,LH공사,2020.06.18,5.963000e+09,5.120864e+09,842135506.0,0.141,6750592.0,4.352587e+06,5.797227e+06,4406871.0,2019.03.30,2019.11.07,2019.11.11,2019.01.16,5400000.0,0.5182,0.4507,0.0126,0.0185,1.0,NaN,서울특별시 강북구 우이동 154-37,2020.06.18,2020.0
44,강북구,우이동,161-1,2종일주,,351.0,1.321400e+09,3764672.0,0.588,다세대주택,659.12,580.32,0.880,188.06,59.99,12개호,1개동,지상5층,LH공사,2019.09.10,3.121000e+09,2.630343e+09,490656608.5,0.157,5378067.0,4.735101e+06,4.532574e+06,4129896.0,2019.02.21,2019.05.08,2019.05.15,2019.09.19,5500000.0,0.5373,0.4298,0.0120,0.0209,1.0,NaN,서울특별시 강북구 우이동 161-1,2019.09.10,2019.0
45,강북구,우이동,23-16,2종일주,4미터,230.0,9.961300e+08,4331000.0,0.494,도생,458.68,385.60,0.841,199.43,59.73,8개호,1개동,지상5층,LH공사,2019.11.12,2.188000e+09,1.918175e+09,269825113.5,0.123,5674274.0,4.770210e+06,4.974520e+06,4330200.0,2019.04.23,2019.07.08,2019.08.05,2019.12.05,5500000.0,0.5548,0.4103,0.0115,0.0235,1.0,NaN,서울특별시 강북구 우이동 23-16,2019.11.12,2019.0
46,강북구,우이동,23-18,2종일주,4미터,264.0,8.000000e+08,3030303.0,0.700,다세대주택,519.89,395.09,0.761,199.50,59.74,10개호,1개동,지상5층,SH공사,2020.08.27,2.327500e+09,1.802421e+09,525078832.5,0.226,5891063.0,4.485536e+06,4.562052e+06,3859439.0,2019.10.29,2020.01.31,2020.02.07,2020.08.10,5500000.0,0.4712,0.4939,0.0138,0.0211,1.0,NaN,서울특별시 강북구 우이동 23-18,2020.08.27,2020.0
47,강북구,우이동,23-86,2종일주,4미터,229.0,9.918000e+08,4331004.0,0.494,도생,456.76,383.68,0.840,199.46,59.92,8개호,1개동,지상5층,LH공사,2019.11.22,2.172000e+09,1.909975e+09,262025285.5,0.121,5660967.0,4.755233e+06,4.978041e+06,4329836.0,2019.04.23,2019.07.08,2019.08.05,2019.12.05,5500000.0,0.5547,0.4103,0.0115,0.0235,1.0,NaN,서울특별시 강북구 우이동 23-86,2019.11.22,2019.0
48,강북구,우이동,28-6,2종일주,4미터,264.0,8.000000e+08,3030303.0,0.731,다세대주택,519.89,395.09,0.865,199.34,58.20,10개호,1개동,지상5층,SH공사,2020.08.27,2.327000e+09,1.691034e+09,635966255.5,0.273,5889797.0,5.094579e+06,4.280123e+06,3571349.0,2019.10.29,2020.01.31,2020.02.07,2020.08.10,5500000.0,0.5022,0.4634,0.0130,0.0214,1.0,NaN,서울특별시 강북구 우이동 28-6,2020.08.27,2020.0
49,강북구,우이동,61-23,2종일주,4미터,349.0,1.527000e+09,4375358.0,0.429,다세대주택,639.69,572.61,1.254,199.34,58.20,10개호,1개동,지상5층,LH공사,2019.04.22,3.121000e+09,2.479897e+09,641103375.5,0.205,5450481.0,6.832910e+06,4.330865e+06,5173492.0,2018.09.15,2020.01.31,2020.02.07,2020.08.10,5500000.0,0.6523,0.3160,0.0088,0.0229,1.0,NaN,서울특별시 강북구 우이동 61-23,2019.04.22,2019.0
